In [1]:
import re
import warnings
warnings.filterwarnings(action='ignore')

In [19]:
def preprocess_data_to_words(filename):
    with open(filename) as f:
        data = f.read()
    data = data.lower()
    words = re.findall(r'\w+', data)
    return words

def count_word_occurence(words):
    word_counts = {}
    for w in words:
        word_counts[w] = word_counts.get(w, 0) + 1
    return word_counts

def word_proba(word_counts):
    word_proba = {}
    unique_words = sum(word_counts.values())
    for w, count in word_counts.items():
        word_proba[w] = count / unique_words
    return word_proba

def edit_delete(word, pos):
    return word[:pos] + word[pos + 1:]

def edit_switch(word, pos1, pos2):
    if len(word) > 1:
        word_list = list(word)
        word_list[pos1], word_list[pos2] = word_list[pos2], word_list[pos1]
        return ''.join(word_list)
    return word

def edit_replace(word, pos, l):
    return word[:pos] + l + word[pos + 1:]

def one_edit(word):
    edits = set()
    for pos in range(len(word)):
        edits.add(edit_delete(word, pos))
    for pos1 in range(len(word) - 1):
        edits.add(edit_switch(word, pos1, pos1 + 1))
    for pos in range(len(word)):
        for l in 'abcdefghijklmnopqrstuvwxyz':
            edits.add(edit_replace(word, pos, l))
    return edits

def two_edits(word):
    two_edit_set = set()
    for w in one_edit(word):
        two_edit_set.update(one_edit(w))
    return two_edit_set

def top_corrections(incorrect_word, vocab, word_probabilities, top_k):
    suggestions = set()
    
    if incorrect_word in vocab:
        suggestions.add(incorrect_word)
    
    suggestions.update(one_edit(incorrect_word).intersection(vocab))
    suggestions.update(two_edits(incorrect_word).intersection(vocab))
    
    best_words = {s: word_probabilities.get(s, 0) for s in suggestions}
    
    top_suggestions = sorted(best_words.items(), key=lambda x: x[1], reverse=True)[:top_k]
    
    return top_suggestions

def min_edit_distance(start_str, end_str, insert_cost=1, delete_cost=1, replace_cost=2):
    start_len = len(start_str)
    end_len = len(end_str)
    cost_matrix = np.zeros((start_len+1, end_len+1))

    for i in range(1, start_len+1):
        cost_matrix[i, 0] = cost_matrix[i-1, 0] + delete_cost

    for j in range(1, end_len+1):
        cost_matrix[0, j] = cost_matrix[0, j-1] + insert_cost

    for i in range(1, start_len+1):
        for j in range(1, end_len+1):
            
           
            if start_str[i-1] == end_str[j-1]:
                diag_cost=cost_matrix[i-1, j-1]
            else:
                diag_cost=cost_matrix[i-1, j-1]+replace_cost
            cost_matrix[i, j] = min(cost_matrix[i-1, j] + delete_cost, 
                                    cost_matrix[i, j-1] + insert_cost, 
                                    diag_cost)

    
    return cost_matrix, cost_matrix[start_len, end_len]

In [17]:
words = preprocess_data_to_words('shakespeare.txt')
word_counts = count_word_occurence(words)
word_probabilities = word_proba(word_counts)

edit_del = [edit_delete(w, pos) for w in word_counts.keys() if len(w) > 1 for pos in range(len(w))]
edit_switchs = [edit_switch(w, pos1, pos2) for w in word_counts.keys() for pos1 in range(len(w)) for pos2 in range(pos1 + 1, len(w))]
edit_rep = [edit_replace(w, pos1, l) for w in word_counts.keys() for pos1 in range(len(w)) for l in 'abcdefghijklmnopqrstuvwxyz']

corrections = top_corrections('dys', set(word_counts.keys()), word_probabilities, top_k=2)
print(corrections)

[('my', 0.01600328272466147), ('is', 0.010389077479762749)]
